In [1]:
import pandas as pd
from sklearn import model_selection
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
import numpy as np
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import re
from datetime import datetime, timedelta

In [6]:
#欠缺部分：处理极端值;手机型号特征待处理

1.数据集划分

In [2]:
'''
def datasplit(route):
    data = pd.read_csv(route,delimiter='\t')
    data.set_index(['sid'],inplace=True)
    y = data.iloc[:]['label']
    x = data.drop(['label'],axis=1)
    x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size=0.2,random_state=42)
    feature = x_train.append(x_test)
    return feature,x_train,x_test,y_train,y_test
'''
def datasplit(routetrain,routetest):
    train = pd.read_csv(routetrain,delimiter='\t')
    test = pd.read_csv(routetest,delimiter='\t')
    train.set_index(['sid'],inplace=True)
    test.set_index(['sid'],inplace=True)
    train_y = train.iloc[:]['label']
    train_x = train.drop(['label'],axis=1)
    feature = train_x.append(test)
    return feature,train_y,train_x,test

In [3]:
feature,train_y,train_x,test = datasplit(routetrain='round1_iflyad_anticheat_traindata.txt',routetest='round1_iflyad_anticheat_testdata_feature.txt')
#feature,train_x,test_x,train_y,test_y = datasplit(route = 'round1_iflyad_anticheat_traindata.txt')



2、缺失值处理

考虑中：对设备宽的处理，因为可以由画图得知，大部分值在2400以内，所以高于此处的应为异常值，把高于2500的值都拉回2500的范围内
_w['w'][_w['w']>2500] = 2500

去掉size特征

分类变量

In [7]:
feature['time'] = pd.to_datetime(feature['nginxtime'] * 1e+6) + timedelta(hours=8)
feature['day'] = feature['time'].dt.dayofyear
feature['hour'] = feature['time'].dt.hour

In [16]:
feature['day'].value_counts()

160    150000
159    150000
158    150000
157    150000
156    150000
155    150000
161    100000
154    100000
Name: day, dtype: int64

In [42]:
feature['ver'] = feature['ver'].fillna(str(-1))
#city,关联省份之后，发现省份居然打码，按照省份分组的众数填充城市
feature['city'] = feature.groupby('province')['city'].transform(lambda x:x.fillna(method = 'ffill'))
feature['model'] = feature['model'].fillna(str(-1))
feature['make'] = feature['make'].fillna(str(-1))
feature['osv'] = feature['osv'].fillna(feature['osv'].mode()[0])
#语言那一列，缺失值很多，大部分值都为中文，可整列剔除

In [43]:
feature['province'].value_counts()

-1.0     1001057
 2.0       33169
 31.0      28287
 3.0       19897
 27.0      16876
 33.0        437
 34.0        159
 32.0        118
Name: province, dtype: int64

特征处理

In [44]:
#feature = feature.drop(['lan','h','w','os','province','idfamd5','model','openudidmd5',],axis=1)
feature = feature.drop(['lan','os','idfamd5','openudidmd5','time','nginxtime',],axis=1)

In [27]:
#数值型特征处理
#使用skew()方法，计算所有整型和浮点型数据列中，数据分布的偏度（skewness）。
#异常值处理


In [45]:
skew_feature = feature[['ppi','nginxtime']].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_feature[skew_feature>0.5]
skew_index = high_skew.index
for i in skew_index:
    feature[i] = boxcox1p(feature[i], boxcox_normmax(feature[i] + 1))

In [46]:
#分类变量特征处理
feature['orientation'] = feature['orientation'].replace([2.0,90.0],1.0)
feature['carrier'] = feature['carrier'].replace(-1.0,0.0)
feature['ntt'] = feature['ntt'].replace(7.0,0.0)

In [47]:
feature['make'] = feature['make'].replace(['Vivo','vivo,V1813A,PD1813C','vivo%20X20Plus','v1816a','VIVO','vivo,vivo Y35A,PD1502A','vivo V1809T','vivoV1813A','vivo,V1818CA,PD1818E','vivo,vivo Y22L,msm8226',
                                           'vivo V3M A','vivo__vivo__vivoY51A','vivoXplay5A',
                                           '😁蓝色天边波花👾亦是心中飞画😚我就问大家一句:我帅不帅😄','vivo+x20'],'vivo')
feature['make'] = feature['make'].replace(['oppo','OPPO%20R7sm','OPPOOPPO A59s','OPPO%20R11%20Pluskt',
                                          'OPPOA59s','OPPO+A37m','OPPO,PCCM00,PCCM00','OPPO+R11',
                                           'OPPOOPPOA57t','oppoa79k'],'OPPO')
feature['make'] = feature['make'].replace(['HUAWEI,CHM-CL00,CHM-CL00','JSN-AL00','HONOR','荣耀','HMA-AL00','EML-AL00',
                                           '华为','LYA-AL00','INE-AL00','CLT-AL00','huawei','HUAWEICLT-AL00','honor',
                                          'HUAWEI__HUAWEI__WAS-AL00','HUAWEI,MHA-TL00,MHA-TL00','JSN-AL00a','ANE-AL00',
                                           'HUAWEI__HUAWEI__EVR-AL00','HUAWEITRT-AL00A','LLD-AL20','COR-AL10','Huawei',
                                           'PAR-AL00','JKM-AL00b',
                                           'COL-AL10','HUAWEI CAZ-TL10','VCE-AL00','PCT-AL10','ELE-AL00',
                                          ],'HUAWEI')
feature['make'] = feature['make'].replace(['xiaomi','Xiaomi,Redmi 6A,cactus','Redmi%204A','Redmi%204A','Xiaomi,MI 9 SE,grus','小米'],'Xiaomi')
feature['make'] = feature['make'].replace(['魅族','M721C','PRO7H','meizu'],'Meizu')
feature['make'] = feature['make'].replace(['三星','SM-J7108','SM-G5108Q'],'samsung')
feature['make'] = feature['make'].replace(['SMARTISAN','锤子','smartisan__SMARTISAN__YQ601',],'smartisan')
feature['make'] = feature['make'].replace(['金立','GiONEE',
                                           'gionee','SM-G5108Q'],'GIONEE')
feature['make'] = feature['make'].replace(['360'],'360手机')
feature['make'] = feature['make'].replace(['zhongxing','zte','中兴'],'ZTE')
feature['make'] = feature['make'].replace(['努比亚'],'nubia')
feature['make'] = feature['make'].replace(['酷派'],'Coolpad')
feature['make'] = feature['make'].replace(['一加'],'OnePlus')
feature['make'] = feature['make'].replace(['Letv','LeEco','LeMobileLeX620'],'LeMobile')
feature['make'] = feature['make'].replace(['LENOVO'],'lenovo')
feature['make'] = feature['make'].replace(['HMD Global'],'Nokia')
feature['make'] = feature['make'].replace(['美图'],'Meitu')


In [48]:
#labelencoder
encoder = ['apptype', 'dvctype', 'ntt', 'orientation', 'osv', 'pkgname', 'ver', 'adunitshowid', 'mediashowid',
           'ip', 'city', 'reqrealip', 'adidmd5','model',''
           'imeimd5', 'macmd5', 'make',]
col_encoder = LabelEncoder()
for feat in encoder:
    col_encoder.fit(feature[feat])
    feature[feat] = col_encoder.transform(feature[feat])

一部分变量做哑变量，IP地址类，上网检索特征处理

In [49]:
train_x = feature[:len(train_x)]
test_x = feature[len(train_x):]

In [50]:
clf2 = RandomForestClassifier(n_estimators=200)
clf2.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [51]:
pre_2 = clf2.predict(test_x)
pre_2 = pd.DataFrame(pre_2)
pre_2.set_index(test.index,inplace=True)
pre_2.to_csv('ans4.csv')

In [53]:
feature['province'].value_counts()

-1.0     1001057
 2.0       33169
 31.0      28287
 3.0       19897
 27.0      16876
 33.0        437
 34.0        159
 32.0        118
Name: province, dtype: int64

In [29]:
clf2 = RandomForestClassifier(n_estimators=200)
clf2.fit(train_x,train_y)
pre_2 = clf2.predict(test_x)
acc_2 = f1_score(test_y,pre_2)

0.8124741842213961

In [ ]:
clf2 = RandomForestClassifier(n_estimators=300)
clf2.fit(train_x,train_y)
pre_2 = clf2.predict(test_x)
acc_2 = f1_score(test_y,pre_2)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf3 = GradientBoostingClassifier(n_estimators=200, max_depth=2, min_samples_split=2, learning_rate=0.01)
clf3.fit(train_x,train_y)
pre3 = clf3.predict(test_x)
print(acc_3)

In [ ]:
model = XGBClassifier(learning_rate=0.01)
model.fit(train_x,train_y)
pre = model.predict(test_x)
acc = f1_score(test_y,pre)